In [6]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import os.path as osp
import sys
import time

import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import functional as F
import torchvision as tv
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets

import numpy as np
from PIL import Image
import argparse
from matplotlib import pyplot as plt
import pandas as pd
import cv2 
import pickle as pkl
import random
import tarfile
import collections

In [7]:
if sys.version_info[0] == 2:
    import xml.etree.cElementTree as ET
else:
    import xml.etree.ElementTree as ET

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [14]:
def test_net(save_folder, net, cuda, testset, transform, thresh):
    # dump predictions and assoc. ground truth to text file for now
    filename = save_folder+'test1.txt'
    num_images = len(testset)
    
    for i in range(num_images):
        print('Testing image {:d}/{:d}....'.format(i+1, num_images))
        img = testset.pull_image(i)
        img_id, annotation = testset.pull_anno(i)
        x = torch.from_numpy(transform(img)[0]).permute(2, 0, 1)
        x = Variable(x.unsqueeze(0))

        with open(filename, mode='a') as f:
            f.write('\nGROUND TRUTH FOR: '+img_id+'\n')
            for box in annotation:
                f.write('label: '+' || '.join(str(b) for b in box)+'\n')
        if cuda:
            x = x.cuda()

        y = net(x)      # forward pass
        detections = y.data
        # scale each detection back up to the image
        scale = torch.Tensor([img.shape[1], img.shape[0],
                             img.shape[1], img.shape[0]])
        pred_num = 0
        for i in range(detections.size(1)):
            j = 0
            while detections[0, i, j, 0] >= 0.6:
                if pred_num == 0:
                    with open(filename, mode='a') as f:
                        f.write('PREDICTIONS: '+'\n')
                score = detections[0, i, j, 0]
                label_name = labelmap[i-1]
                pt = (detections[0, i, j, 1:]*scale).cpu().numpy()
                coords = (pt[0], pt[1], pt[2], pt[3])
                pred_num += 1
                with open(filename, mode='a') as f:
                    f.write(str(pred_num)+' label: '+label_name+' score: ' +
                            str(score) + ' '+' || '.join(str(c) for c in coords) + '\n')
                j += 1

In [18]:
from data import VOC_ROOT, VOC_CLASSES as labelmap
def test_voc():
    
    # load net
    num_classes = len(VOC_CLASSES) + 1 # +1 background
    net = build_ssd('test', 300, num_classes) # initialize SSD
    net.load_state_dict(torch.load(trained_model))
    
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    
    net.eval()
    print('Finished loading model!')
    
    # load data
    testset = VOCDetection(voc_root, [('2012', 'val')], None, VOCAnnotationTransform())
    
    if cuda:
        net = net.cuda()
        cudnn.benchmark = True
     
    # evaluation
    test_net(save_folder, net, cuda, testset,
             BaseTransform(net.size, (104, 117, 123)),
             thresh=visual_threshold)
    print("/nTesting Done !!")

In [16]:
data_set = 'VOC'
trained_model = 'ssd300_mAP_77.43_v2.pth'
save_folder = 'eval/'
visual_threshold = 0.6
cuda = True
voc_root = '//datasets/ee285f-public/PascalVOC2012/'

In [17]:
test_voc()

Finished loading model!
Testing image 1/5823....
Testing image 2/5823....
Testing image 3/5823....
Testing image 4/5823....
Testing image 5/5823....
Testing image 6/5823....
Testing image 7/5823....
Testing image 8/5823....
Testing image 9/5823....
Testing image 10/5823....
Testing image 11/5823....
Testing image 12/5823....
Testing image 13/5823....
Testing image 14/5823....
Testing image 15/5823....
Testing image 16/5823....
Testing image 17/5823....
Testing image 18/5823....
Testing image 19/5823....
Testing image 20/5823....
Testing image 21/5823....
Testing image 22/5823....
Testing image 23/5823....
Testing image 24/5823....
Testing image 25/5823....
Testing image 26/5823....
Testing image 27/5823....
Testing image 28/5823....
Testing image 29/5823....
Testing image 30/5823....
Testing image 31/5823....
Testing image 32/5823....
Testing image 33/5823....
Testing image 34/5823....
Testing image 35/5823....
Testing image 36/5823....
Testing image 37/5823....
Testing image 38/5823..

Testing image 311/5823....
Testing image 312/5823....
Testing image 313/5823....
Testing image 314/5823....
Testing image 315/5823....
Testing image 316/5823....
Testing image 317/5823....
Testing image 318/5823....
Testing image 319/5823....
Testing image 320/5823....
Testing image 321/5823....
Testing image 322/5823....
Testing image 323/5823....
Testing image 324/5823....
Testing image 325/5823....
Testing image 326/5823....
Testing image 327/5823....
Testing image 328/5823....
Testing image 329/5823....
Testing image 330/5823....
Testing image 331/5823....
Testing image 332/5823....
Testing image 333/5823....
Testing image 334/5823....
Testing image 335/5823....
Testing image 336/5823....
Testing image 337/5823....
Testing image 338/5823....
Testing image 339/5823....
Testing image 340/5823....
Testing image 341/5823....
Testing image 342/5823....
Testing image 343/5823....
Testing image 344/5823....
Testing image 345/5823....
Testing image 346/5823....
Testing image 347/5823....
T

Testing image 616/5823....
Testing image 617/5823....
Testing image 618/5823....
Testing image 619/5823....
Testing image 620/5823....
Testing image 621/5823....
Testing image 622/5823....
Testing image 623/5823....
Testing image 624/5823....
Testing image 625/5823....
Testing image 626/5823....
Testing image 627/5823....
Testing image 628/5823....
Testing image 629/5823....
Testing image 630/5823....
Testing image 631/5823....
Testing image 632/5823....
Testing image 633/5823....
Testing image 634/5823....
Testing image 635/5823....
Testing image 636/5823....
Testing image 637/5823....
Testing image 638/5823....
Testing image 639/5823....
Testing image 640/5823....
Testing image 641/5823....
Testing image 642/5823....
Testing image 643/5823....
Testing image 644/5823....
Testing image 645/5823....
Testing image 646/5823....
Testing image 647/5823....
Testing image 648/5823....
Testing image 649/5823....
Testing image 650/5823....
Testing image 651/5823....
Testing image 652/5823....
T

Testing image 921/5823....
Testing image 922/5823....
Testing image 923/5823....
Testing image 924/5823....
Testing image 925/5823....
Testing image 926/5823....
Testing image 927/5823....
Testing image 928/5823....
Testing image 929/5823....
Testing image 930/5823....
Testing image 931/5823....
Testing image 932/5823....
Testing image 933/5823....
Testing image 934/5823....
Testing image 935/5823....
Testing image 936/5823....
Testing image 937/5823....
Testing image 938/5823....
Testing image 939/5823....
Testing image 940/5823....
Testing image 941/5823....
Testing image 942/5823....
Testing image 943/5823....
Testing image 944/5823....
Testing image 945/5823....
Testing image 946/5823....
Testing image 947/5823....
Testing image 948/5823....
Testing image 949/5823....
Testing image 950/5823....
Testing image 951/5823....
Testing image 952/5823....
Testing image 953/5823....
Testing image 954/5823....
Testing image 955/5823....
Testing image 956/5823....
Testing image 957/5823....
T

Testing image 1218/5823....
Testing image 1219/5823....
Testing image 1220/5823....
Testing image 1221/5823....
Testing image 1222/5823....
Testing image 1223/5823....
Testing image 1224/5823....
Testing image 1225/5823....
Testing image 1226/5823....
Testing image 1227/5823....
Testing image 1228/5823....
Testing image 1229/5823....
Testing image 1230/5823....
Testing image 1231/5823....
Testing image 1232/5823....
Testing image 1233/5823....
Testing image 1234/5823....
Testing image 1235/5823....
Testing image 1236/5823....
Testing image 1237/5823....
Testing image 1238/5823....
Testing image 1239/5823....
Testing image 1240/5823....
Testing image 1241/5823....
Testing image 1242/5823....
Testing image 1243/5823....
Testing image 1244/5823....
Testing image 1245/5823....
Testing image 1246/5823....
Testing image 1247/5823....
Testing image 1248/5823....
Testing image 1249/5823....
Testing image 1250/5823....
Testing image 1251/5823....
Testing image 1252/5823....
Testing image 1253/5

...

Testing image 4454/5823....
Testing image 4455/5823....
Testing image 4456/5823....
Testing image 4457/5823....
Testing image 4458/5823....
Testing image 4459/5823....
Testing image 4460/5823....
Testing image 4461/5823....
Testing image 4462/5823....
Testing image 4463/5823....
Testing image 4464/5823....
Testing image 4465/5823....
Testing image 4466/5823....
Testing image 4467/5823....
Testing image 4468/5823....
Testing image 4469/5823....
Testing image 4470/5823....
Testing image 4471/5823....
Testing image 4472/5823....
Testing image 4473/5823....
Testing image 4474/5823....
Testing image 4475/5823....
Testing image 4476/5823....
Testing image 4477/5823....
Testing image 4478/5823....
Testing image 4479/5823....
Testing image 4480/5823....
Testing image 4481/5823....
Testing image 4482/5823....
Testing image 4483/5823....
Testing image 4484/5823....
Testing image 4485/5823....
Testing image 4486/5823....
Testing image 4487/5823....
Testing image 4488/5823....
Testing image 4489/5

Testing image 4748/5823....
Testing image 4749/5823....
Testing image 4750/5823....
Testing image 4751/5823....
Testing image 4752/5823....
Testing image 4753/5823....
Testing image 4754/5823....
Testing image 4755/5823....
Testing image 4756/5823....
Testing image 4757/5823....
Testing image 4758/5823....
Testing image 4759/5823....
Testing image 4760/5823....
Testing image 4761/5823....
Testing image 4762/5823....
Testing image 4763/5823....
Testing image 4764/5823....
Testing image 4765/5823....
Testing image 4766/5823....
Testing image 4767/5823....
Testing image 4768/5823....
Testing image 4769/5823....
Testing image 4770/5823....
Testing image 4771/5823....
Testing image 4772/5823....
Testing image 4773/5823....
Testing image 4774/5823....
Testing image 4775/5823....
Testing image 4776/5823....
Testing image 4777/5823....
Testing image 4778/5823....
Testing image 4779/5823....
Testing image 4780/5823....
Testing image 4781/5823....
Testing image 4782/5823....
Testing image 4783/5

Testing image 5041/5823....
Testing image 5042/5823....
Testing image 5043/5823....
Testing image 5044/5823....
Testing image 5045/5823....
Testing image 5046/5823....
Testing image 5047/5823....
Testing image 5048/5823....
Testing image 5049/5823....
Testing image 5050/5823....
Testing image 5051/5823....
Testing image 5052/5823....
Testing image 5053/5823....
Testing image 5054/5823....
Testing image 5055/5823....
Testing image 5056/5823....
Testing image 5057/5823....
Testing image 5058/5823....
Testing image 5059/5823....
Testing image 5060/5823....
Testing image 5061/5823....
Testing image 5062/5823....
Testing image 5063/5823....
Testing image 5064/5823....
Testing image 5065/5823....
Testing image 5066/5823....
Testing image 5067/5823....
Testing image 5068/5823....
Testing image 5069/5823....
Testing image 5070/5823....
Testing image 5071/5823....
Testing image 5072/5823....
Testing image 5073/5823....
Testing image 5074/5823....
Testing image 5075/5823....
Testing image 5076/5

Testing image 5334/5823....
Testing image 5335/5823....
Testing image 5336/5823....
Testing image 5337/5823....
Testing image 5338/5823....
Testing image 5339/5823....
Testing image 5340/5823....
Testing image 5341/5823....
Testing image 5342/5823....
Testing image 5343/5823....
Testing image 5344/5823....
Testing image 5345/5823....
Testing image 5346/5823....
Testing image 5347/5823....
Testing image 5348/5823....
Testing image 5349/5823....
Testing image 5350/5823....
Testing image 5351/5823....
Testing image 5352/5823....
Testing image 5353/5823....
Testing image 5354/5823....
Testing image 5355/5823....
Testing image 5356/5823....
Testing image 5357/5823....
Testing image 5358/5823....
Testing image 5359/5823....
Testing image 5360/5823....
Testing image 5361/5823....
Testing image 5362/5823....
Testing image 5363/5823....
Testing image 5364/5823....
Testing image 5365/5823....
Testing image 5366/5823....
Testing image 5367/5823....
Testing image 5368/5823....
Testing image 5369/5

Testing image 5627/5823....
Testing image 5628/5823....
Testing image 5629/5823....
Testing image 5630/5823....
Testing image 5631/5823....
Testing image 5632/5823....
Testing image 5633/5823....
Testing image 5634/5823....
Testing image 5635/5823....
Testing image 5636/5823....
Testing image 5637/5823....
Testing image 5638/5823....
Testing image 5639/5823....
Testing image 5640/5823....
Testing image 5641/5823....
Testing image 5642/5823....
Testing image 5643/5823....
Testing image 5644/5823....
Testing image 5645/5823....
Testing image 5646/5823....
Testing image 5647/5823....
Testing image 5648/5823....
Testing image 5649/5823....
Testing image 5650/5823....
Testing image 5651/5823....
Testing image 5652/5823....
Testing image 5653/5823....
Testing image 5654/5823....
Testing image 5655/5823....
Testing image 5656/5823....
Testing image 5657/5823....
Testing image 5658/5823....
Testing image 5659/5823....
Testing image 5660/5823....
Testing image 5661/5823....
Testing image 5662/5